# Постановка задачі

Створити, навчити і апробувати багатошарову нейронну мережу з прямою передачею сигналу для ухвалення рішення про зарахування до Університету абітурієнтів, які здали вступні іспити з математики, англійської та української мови.

Правила прийому наступні:
1. Рейтинг абітурієнтів формується за формулою 0,4 БМ+0,3БА+0,3БУ, де БМ-бал з іспиту з математики, БА-бал з іспиту з англійської мови, БУ-бал з іспиту з української мови.
2. Мінімальний прохідний бал на вступ 160 для абітурієнтів без пільг.
3. З математики для абітурієнтів без пільг мінімальний бал іспиту не може бути менший 140 балів.
4. Абітурієнти, які мають пільги, зараховуються при мінімумі 120 балів з усіх іспитів і їх рейтинг не може бути меншим ніж 144 бали
5. Університет може прийняти на навчання 350 абітурієнтів, з них не більше 10% це абітурієнти з пільгами.
6. Статистика минулих років показує, що в середньому до Університету подають документи 1500 абітурієнтів.

Для навчання мережі слід використовувати всі вивчені методи адаптації та навчання та провести аналіз їх ефективності. Слід також визначити мінімальну кількість шарів і нейронів, що забезпечує задовільне рішення поставленої задачі. У звіті навести архітектуру мережі та код реалізації.
Результат має бути візуалізований та представлений у формі Еxcel таблиці, як список зарахованих абітурієнтів.

# Виконання

Імпортую бібліотеки

In [1]:
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
# from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
np.random.seed(1)

In [2]:
# pip install keras==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.12.0 which is incompatible.


Створюю константи, які визначатиму параметри даних

In [2]:
num_years = 5 # кількість минулих років
num_all_avg = 1500 # кількість вступників кожного року в середньому
num_places = 350 # кількість місць
part_privilege = 0.1 # відсоток з пільгами

Створюю функції для базових розрахунків

Функція розрахунку пільгових місць

In [3]:
def num_priv(num):
  return int(num * part_privilege)

Функція для розрахунку непільгових місць

In [4]:
def num_unpriv(num):
  return num - int(num * part_privilege)

Функція для генерації оцінок

In [5]:
def randrate(num, minr, maxr, mean, std):
  rates = np.random.normal(mean, std, num)

  rates = pd.Series (map(
    lambda x:
      minr if x < minr else
      maxr if x > maxr else
      int(x), rates))

  return rates

Функція для генерації результатів вступу до університету за рік

In [6]:
def generate_df(seed):
  np.random.seed(seed)
  num_all = int(np.random.normal(num_all_avg, 200))
  df = pd.DataFrame()
  ids = list(range(num_all))
  np.random.shuffle(ids)
  df['id'] = ids
  df['rate_math'] = randrate(num_all, 100, 200, 155, 45)
  df['rate_en'] = randrate(num_all, 100, 200, 155, 45)
  df['rate_ukr'] = randrate (num_all, 100, 200, 155, 45)
  df['rate_res'] = df['rate_math']*0.4 + df['rate_en']*0.3 + df['rate_ukr']*0.3
  is_priv = [1]*num_priv(num_all) + [0]*num_unpriv(num_all)
  np.random.shuffle(is_priv)
  df['is_priv'] = is_priv

  df_priv = df[df['is_priv'] == 1].sort_values('rate_res', ascending=False)
  df_priv_chosen = df_priv[
    (df_priv['rate_en'] >= 120) &
    (df_priv['rate_ukr'] >= 120) &
    (df_priv['rate_math'] >= 120) &
    (df_priv['rate_res'] >= 144)
  ].iloc[0:num_priv(num_places)]['id']


  df_unpriv = df[df['is_priv'] == 0].sort_values('rate_res', ascending=False)
  df_unpriv_chosen = df_unpriv[ (df_unpriv['rate_res'] >= 160) & (df_unpriv['rate_math'] >= 140) ].iloc[0:num_unpriv(num_places)]['id']
  df_chosen = pd.concat([df_priv_chosen, df_unpriv_chosen])
  df.loc[df['id'].isin(df_chosen), 'decision'] = 1
  df.loc[df['decision'] != 1, 'decision'] = 0
  return df

Генерую дані для заданої кількості років

In [7]:
data = [None]*num_years
for year in range(num_years):
  data[year] = generate_df(year)

df = data[0]
df[df['decision'] == 1].sort_values('rate_res')

,id,rate_math,rate_en,rate_ukr,rate_res,is_priv,decision
1126,1154,168,154,182,168.0,1,1.0
1238,671,145,168,199,168.1,1,1.0
725,1769,146,167,200,168.5,1,1.0
483,1347,152,200,159,168.5,1,1.0
1640,928,162,167,179,168.6,1,1.0
...,...,...,...,...,...,...,...
652,1178,200,198,200,199.4,0,1.0
1818,850,200,200,200,200.0,0,1.0
562,1761,200,200,200,200.0,0,1.0
281,458,200,200,200,200.0,0,1.0


Готую розмічені дані, ділю їх на x та y

In [8]:
X = pd.DataFrame()
y = pd.DataFrame()

for df in data:
  X = pd.concat(( X, df[['rate_math', 'rate_en', 'rate_ukr', 'is_priv']] ))
  y = pd.concat(( y, df[['decision']] ))

Переводжу значення в інтервал від 0 до 1

In [9]:
X[['rate_math', 'rate_en', 'rate_ukr']] = (X[['rate_math', 'rate_en', 'rate_ukr']] - 100).astype('float32')/100

Розділяю дані на навчальні та тестові

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Визначаю тип моделі та шари

In [11]:
model = models.Sequential()

model.add(layers.Dense(16, activation='relu', input_dim=4))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Компілюю модель

In [12]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Навчаю модель

In [13]:
model.fit(X_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
677/677 [==============================] - 2s 2ms/step - loss: 0.4459 - accuracy: 0.8060
Epoch 2/10
677/677 [==============================] - 2s 3ms/step - loss: 0.2043 - accuracy: 0.9273
Epoch 3/10
677/677 [==============================] - 1s 2ms/step - loss: 0.1254 - accuracy: 0.9568
Epoch 4/10
677/677 [==============================] - 1s 2ms/step - loss: 0.1005 - accuracy: 0.9631
Epoch 5/10
677/677 [==============================] - 1s 2ms/step - loss: 0.0898 - accuracy: 0.9641
Epoch 6/10
677/677 [==============================] - 1s 2ms/step - loss: 0.0830 - accuracy: 0.9662
Epoch 7/10
677/677 [==============================] - 2s 3ms/step - loss: 0.0777 - accuracy: 0.9675
Epoch 8/10
677/677 [==============================] - 2s 2ms/step - loss: 0.0757 - accuracy: 0.9684
Epoch 9/10
677/677 [==============================] - 1s 2ms/step - loss: 0.0738 - accuracy: 0.9665
Epoch 10/10
677/677 [==============================] - 1s 2ms/step - loss: 0.0729 - accuracy: 0.9681

Визначаю мінімальну кількість нейронів та шарів, що забезпечують точність не менше 90%

Фукція, у якій буде створюватися та компілюватися модель

In [21]:
def create_model(num_layers=1, num_neurons=1):
    model = models.Sequential()
    model.add(layers.Dense(num_neurons, activation='relu', input_dim=4))
    for _ in range(num_layers - 1):
        model.add(layers.Dense(num_neurons, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

Функція, яка розділяє вибірку на навчальні та тестові дані, задає певні параметри та шукає мінімалоно необхідну модель

In [32]:
from keras.wrappers.scikit_learn import KerasClassifier

def find_minimal_model(X, y, target_accuracy=0.90):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    for num_layers in range(1, 3):
        for num_neurons in range(1, 17):
            model = create_model(num_layers=num_layers, num_neurons=num_neurons)
            model.fit(X_train, y_train, epochs=10, batch_size=10, verbose=0)
            y_pred = model.predict(X_test).round()
            accuracy = accuracy_score(y_test, y_pred)
            print(f"Layers: {num_layers}, Neurons: {num_neurons}, Accuracy: {accuracy}")
            if accuracy >= target_accuracy:
                return num_layers, num_neurons, accuracy
    return None, None, None

Викликаю функцію пошуку мінімальних параметрів та виводжу результат

In [34]:
best_layers, best_neurons, best_accuracy = find_minimal_model(X_train, y_train)
print(f"Minimal model - Layers: {best_layers}, Neurons: {best_neurons}, Accuracy: {best_accuracy}")

43/43 [==============================] - 0s 2ms/step
Layers: 1, Neurons: 1, Accuracy: 0.7998522895125554
43/43 [==============================] - 0s 1ms/step
Layers: 1, Neurons: 2, Accuracy: 0.9608567208271788
Minimal model - Layers: 1, Neurons: 2, Accuracy: 0.9608567208271788


Визначаю рішення про зарахування студента

In [38]:
y_pred = model.predict(X_test, verbose=0).round()

map_result = {
  0: 'не зарах.',
  1: 'зарах.'
}

map_priv = {
  0: 'нi',
  1: 'так'
}

rate_math = X_test['rate_math']*100+100
rate_ukr = X_test['rate_ukr' ]*100+100
rate_en = X_test['rate_en']*100+100
rate = 0.4*rate_math + 0.3*rate_en + 0.3*rate_ukr

rate_math = rate_math.reset_index(drop=True)
rate_ukr = rate_ukr.reset_index(drop=True)
rate_en = rate_en.reset_index(drop=True)
is_priv = X_test['is_priv'].map(map_priv).reset_index(drop=True)
result = pd.Series(y_pred.flatten()).map(map_result).reset_index(drop=True)
rate = rate.reset_index(drop=True)

df_res = pd.DataFrame.from_dict({
  'Оцінка з математики': rate_math,
  'Оцінка з укр мови': rate_ukr,
  'Оцінка з англ мови': rate_en,
  'Подається на пільгове місце?': is_priv,
  'Рішення моделі': result,
  'Рейтинг': rate
})
df_res = df_res.sort_values('Рейтинг', ascending=False)


n_priv = len(df_res[(df_res['Подається на пільгове місце?']=='так') & (df_res['Рішення моделі']=='зарах.')])
n_yes = len(df_res[(df_res['Рішення моделі']=='зарах.')])

df_res.to_excel('Результат.xlsx', index=False)

accuracy = accuracy_score(y_test, y_pred)
print(f'Toчнicть: {accuracy}')
print(f'Кількість зарахованих: {n_yes}')
print(f'Кількість пільгових: {n_priv}')

Toчнicть: 0.9739952718676123
Кількість зарахованих: 334
Кількість пільгових: 31


# Висновок

У цій роботі я створив, навчив та апробував багатошарову нейронну
мережу з прямою передачею сигналу для ухвалення рішення про
зарахування до Університету абітурієнтів, які здали вступні іспити з
математики, англійської та української мови. Визначив мінімальну кількість шарів і нейронів, що забезпечує задовільне рішення
поставленої задачі. Візуалізував та представив результат у формі Еxcel таблиці, як список зарахованих абітурієнтів.